## Import libraries

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import os
from skimage import io
import pandas as pd
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from skimage.segmentation import mark_boundaries
from skimage.measure import regionprops, label
import SimpleITK as sitk
from scipy.ndimage import binary_fill_holes
from scipy import ndimage as ndi
from skimage.morphology import closing, square, remove_small_objects, binary_erosion, disk, binary_dilation
from skimage.segmentation import clear_border
from skimage.color import label2rgb
from time import time
from skimage.filters import  threshold_otsu, threshold_triangle, gaussian, threshold_local
from skimage.morphology import convex_hull_image
from skimage.draw import line, polygon

print("Libraries imported") # from mpl_toolkits.mplot3d import Axes3D

Libraries imported


### Define functions to be used

In [2]:
def show_gray_image (img):
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.imshow(img, cmap=plt.cm.gray)
    ax.set_axis_off()
    plt.tight_layout()
    plt.show()

In [3]:
def segment_and_label_dapi(dapi_image, debris_size, min_area_to_keep_cell, erosion_radius, block_size = 251):
    ''' Segments the dapi image.
    Ipnuts:
    
    dapi_image: 2-dimensional numpy array (image array)
    debris_size: size of the debris to be removed (in pixels)
    erosion_radius: radius (in pixels) of the disk to be used for binary ersion to separate connected nuclei
    
    Outputs:
    nuclear_mask: segmented nuclei image
    nuclear_labels: labled nuclei
    
    '''
    # locad thresholding to segment the nuclei
    adaptive_thresh = threshold_local(dapi_image, block_size = block_size)
    seg_dapi = dapi_image > adaptive_thresh
    
    # use Otsu method to segment the dapi image
#     seg_dapi = dapi_image > threshold_otsu(dapi_image)

    # fill the holes in the image
    nuclear_mask = ndi.binary_fill_holes(seg_dapi)
    
    # remove small debris from the segmented imaage
    seg_dapi = remove_small_objects(seg_dapi, debris_size)

    
    
    nuclear_mask = remove_small_objects(nuclear_mask, min_area_to_keep_cell)
    
    # erode the image
    eroded_mask = binary_erosion(nuclear_mask, disk(erosion_radius))

    distance = ndi.distance_transform_edt(eroded_mask)
    local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((1, 1)),
                                labels=nuclear_mask)
    markers = ndi.label(local_maxi)[0]
    nuclear_labels = watershed(-distance, markers, mask=nuclear_mask, connectivity=2)
    
    return nuclear_mask, nuclear_labels
    

In [4]:
def obtain_big_cells(labeled_img, area_thresh, cir_thresh):
    circ = lambda r: (4 * np.pi * r.area) / (r.perimeter * r.perimeter)
    big_cells = [(prop.label, prop.area, circ(prop)) for prop in regionprops(labeled_img)
                 if prop.area > area_thresh and circ(prop) < cir_thresh]
    return big_cells

In [5]:
def split_cells(label_rem, image):
    
    relabels, tmp_img = np.zeros_like(image), np.zeros_like(image)
    pts = []
    row_cords =[]
    col_cords = []
    for prop in regionprops(label_rem):
        x,y = np.int16(np.round(prop.centroid))
        row_cords.append(x)
        col_cords.append(y)
        pts.extend([x,y])
    if len(regionprops(label_rem))>2:
        rr, cc = polygon(row_cords, col_cords)
    else:
        rr, cc = line(pts[0], pts[1], pts[2], pts[3])

    tmp_img[rr,cc] =1
    tmp_img = binary_dilation(tmp_img, disk(10))
    tmp_img = convex_hull_image(tmp_img)
    split_img = np.logical_and(image, np.logical_not(tmp_img))
    distance_rem = ndi.distance_transform_edt(split_img)
    local_maxi_rem = peak_local_max(distance_rem, indices=False, footprint=np.ones((1, 1)),
                                labels=image)
    markers_rem = label(local_maxi_rem, connectivity=2)
    labels_rem = watershed(-distance_rem, markers_rem, mask=image, connectivity=2)
    return labels_rem
        

In [6]:
def obtain_final_labels_after_splitting_big_objects(nuclear_labels, big_cells, 
                                                    ero_rad, min_rem_area, min_area_to_keep_cell):
    '''
    Inputs:
    
    '''
    
    relabels = np.zeros_like(nuclear_labels)

    for ii, area, cir in big_cells:
        image = nuclear_labels==ii
        chull = convex_hull_image(image)
        rem = np.logical_and(chull, np.logical_not(image))
        eroded_rem = binary_erosion(rem, disk(ero_rad))
        eroded_rem = remove_small_objects(eroded_rem, min_rem_area)
        label_rem = label(eroded_rem)
    #         print(ii, np.max(label_rem))

        if np.max(label_rem) >= 2:
#             show_gray_image(split_cells(label_rem, image))
            relabels = label(relabels + split_cells(label_rem, image))

    assigned_relabels = np.zeros_like(relabels)
    for p in regionprops(relabels):
        if p.label > 0:
            assigned_relabels[np.where(relabels==p.label)] = np.max(nuclear_labels) + p.label

    label_img = label(nuclear_labels + assigned_relabels)
    label_img = remove_small_objects(label_img, min_area_to_keep_cell)
#     final_labels = clear_border(label_img)
    final_labels = label_img
    return final_labels

### Provide root directory containing images 

In [7]:
root_path = r'D:\Jove_Experiments\2020_0313_Jove_MCF7_E2_DV'

In [8]:
fpaths = [os.path.join(root_path,f) for f in os.listdir(root_path) if os.path.isdir(os.path.join(root_path,f))]

In [9]:
features = pd.DataFrame()
for fpath in fpaths:
    f_names = [f for f in os.listdir(fpath) if f.endswith('.tif')]
    for fname in f_names:
        fname_noext = os.path.splitext(fname)[0]
        year, mon_date, _, cell_line, treatment, field, _, _, _, wavelength = fname_noext.split('_')
        # Store features
        features = features.append([{'filename': fname,
                                     'filepath': fpath,
                                     'date': ('').join([year,mon_date]),
                                     'cell_line': cell_line,
                                     'treatment': treatment,
                                      'field' : field,
                                      'wavelength': wavelength,
                                     },])
    
features = features.reset_index()

In [10]:
d = {'w435': 'dapi', 'w594': 'exon',  'w676': 'intron'}

features["wavelength"] = features["wavelength"].map(d)

## User defined parameter

In [14]:
sigma = 1
erosion_radius = 31
debris_size = 100
min_area_to_keep_cell = 3001
dilation_radius = 100
area_thresh =20000 # maximum area of the cell to be considered one cell
cir_thresh = 0.70
ero_rad = 5 # 5 for DV, radius of the disk to be used for eroading the region (convex hull - roi)
min_rem_area = 2 # 2 for DV

In [15]:
seg_dir = os.path.join(os.path.dirname(root_path), 'Results', 'Segmentation_erosion_radius_'+ str(erosion_radius)
                       + '_min_area_' + str(min_area_to_keep_cell) +'_area_thresh_' + str(area_thresh))
if not os.path.exists(seg_dir):
    os.makedirs(seg_dir)

In [13]:
import warnings
warnings.filterwarnings("ignore")

In [16]:
starting_time = time()
summary_df = pd.DataFrame()

for fpath in features['filepath'].unique():
    df = features[features['filepath'] == fpath]
    for channel in df['wavelength']:
        sdf = df[df['wavelength']==channel]
        fname = sdf['filename'].get_values()[0]
        
        if channel == 'dapi':
            dapi_image =  io.imread(os.path.join(fpath, fname))
            # get the nuclear mask and nuclear labels
            nuclear_mask, nuclear_labels = segment_and_label_dapi(dapi_image, debris_size, 
                                                                  min_area_to_keep_cell, erosion_radius)
            
            # get the big cells (to be split if it contains more than 1 cells)
            big_cells = obtain_big_cells(nuclear_labels, area_thresh, cir_thresh)
            
            # get the final labels after splitting the big cells
            final_labels = obtain_final_labels_after_splitting_big_objects(nuclear_labels, big_cells, ero_rad, 
                                                        min_rem_area, min_area_to_keep_cell)
            # nuclei not touching border
            non_border_labels = clear_border(final_labels)
            nuc_mask = final_labels > 0
            
            # dialte the nuclear mask to create cell mask
            temp_img = binary_dilation(nuc_mask, disk(dilation_radius))
            
            # use watershed method with final_labels as markers to obtain labeled cell mask
            cell_labels = watershed(temp_img, markers=final_labels, mask=temp_img)
            
            # mark the boundaries of the cell mask
            marked_img = mark_boundaries(dapi_image, cell_labels, color=(1, 0, 0), outline_color=(0, 1, 0))
            io.imsave(os.path.join(seg_dir, fname), np.uint8(255*nuc_mask))
            io.imsave(os.path.join(seg_dir, 'outline_' + fname), np.uint8(marked_img*255))
            
        elif channel == 'exon':
            exon_image =  io.imread(os.path.join(fpath, fname))
            # segment using Otsu thresholding
            seg_exon = exon_image > threshold_otsu(exon_image)
            
            io.imsave(os.path.join(seg_dir, fname), np.uint8(255*seg_exon))

        elif channel == 'intron':
            intron_image = sitk.ReadImage(os.path.join(fpath, fname))
            gaussian_blur = sitk.SmoothingRecursiveGaussianImageFilter()
            gaussian_blur.SetSigma ( float ( sigma ) )
            blur_intron = gaussian_blur.Execute ( intron_image )

            max_entropy_filter = sitk.MaximumEntropyThresholdImageFilter()
            max_entropy_filter.SetInsideValue(0)
            max_entropy_filter.SetOutsideValue(1)
            seg = max_entropy_filter.Execute(blur_intron)
            seg_intron = sitk.GetArrayFromImage(seg)
#             intron_image =  io.imread(os.path.join(fpath, fname))
#             blur_intron = gaussian(intron_image, sigma = sigma)
#             # segment using Otsu thresholding
#             seg_intron = blur_intron > threshold_otsu(blur_intron)
            
            io.imsave(os.path.join(seg_dir, fname), np.uint8(255*seg_intron))
            
        else:
            print('Different than dapi, exon or intron image found!')
    print(fpath, time()-starting_time, 'seconds')       
    # save the images
    comb_img = np.zeros((seg_exon.shape[0], seg_exon.shape[0], 3))
    comb_img[:,:,0] = seg_intron
    comb_img[:,:,1] = seg_exon
    comb_img[:,:,2] = nuc_mask
    marked_img = mark_boundaries(comb_img, cell_labels, color=(1, 1, 1), outline_color=(1, 1, 1))
    io.imsave(os.path.join(seg_dir, 'combined_' + fname), np.uint8(marked_img*255))


    ### get measurements
    # number of cells not toching border
    cell_count = np.count_nonzero(np.unique(non_border_labels))
    
    for cell_prop in regionprops(non_border_labels):
        
        cell_id = cell_prop.label
        
        cell_region = cell_labels==cell_id

         # label exons in the cell
        cell_exon = label(cell_region*seg_exon)

        # label intons in the cell
        cell_intron = label(cell_region*seg_intron)

        # label bursts in the cell
        cell_burst = label(cell_region*np.logical_and(seg_intron, seg_exon))

        #combine labels
        comb_labs =[('green', cell_exon), ('red', cell_intron), ('yellow', cell_burst)]

        for col, lab in comb_labs:
            for prop in regionprops(lab):
                summary_df = summary_df.append([{'cell_id': cell_id,
                                'cell_area': cell_prop.area,
                                 'cell_centroid': cell_prop.centroid,
                                    'dot_area': prop.area,
                                    'dot_centroid': prop.centroid,
                                    'dot_color' : col,
                                    'field' :  df.iloc[0]['field'],
                                    'treatment' :  df.iloc[0]['treatment'],
                                    'cell_count' : cell_count,
                                 },])

summary_df = summary_df.reset_index()
summary_df.to_excel(os.path.join(seg_dir,'JOVE_exp_summary.xlsx'))
# print('Total time taken is', time()- starting_time, 'seconds')

D:\Jove_Experiments\2020_0313_Jove_MCF7_E2_DV\2020_0313_Jove_MCF7_E2_01_R3D_D3D_PRJ_TIFFS 17.607056856155396 seconds
D:\Jove_Experiments\2020_0313_Jove_MCF7_E2_DV\2020_0313_Jove_MCF7_E2_02_R3D_D3D_PRJ_TIFFS 70.92408227920532 seconds
D:\Jove_Experiments\2020_0313_Jove_MCF7_E2_DV\2020_0313_Jove_MCF7_E2_03_R3D_D3D_PRJ_TIFFS 110.39000272750854 seconds
D:\Jove_Experiments\2020_0313_Jove_MCF7_E2_DV\2020_0313_Jove_MCF7_E2_04_R3D_D3D_PRJ_TIFFS 146.34575247764587 seconds
D:\Jove_Experiments\2020_0313_Jove_MCF7_E2_DV\2020_0313_Jove_MCF7_E2_05_R3D_D3D_PRJ_TIFFS 191.4196743965149 seconds
D:\Jove_Experiments\2020_0313_Jove_MCF7_E2_DV\2020_0313_Jove_MCF7_E2_06_R3D_D3D_PRJ_TIFFS 235.47303986549377 seconds
D:\Jove_Experiments\2020_0313_Jove_MCF7_E2_DV\2020_0313_Jove_MCF7_E2_07_R3D_D3D_PRJ_TIFFS 303.4899685382843 seconds
D:\Jove_Experiments\2020_0313_Jove_MCF7_E2_DV\2020_0313_Jove_MCF7_E2_08_R3D_D3D_PRJ_TIFFS 353.80703139305115 seconds
D:\Jove_Experiments\2020_0313_Jove_MCF7_E2_DV\2020_0313_Jove_MCF7_E

In [17]:
conditions = summary_df['treatment'].unique()
requirements = [('green', 0), ('red', 9), ('yellow', 4)] # ('green', 0): dot color and dot area threshold

In [18]:
summary = pd.DataFrame()
for condition in conditions:
    for dot_col, dot_area_thres in requirements:
        num_cells, num_dots =0, 0

        for field in summary_df['field'].unique():
            df = summary_df[(summary_df['field'] == field) & 
                            (summary_df['treatment'] == condition) &
                           (summary_df['dot_color'] == dot_col)]
        #     print(field, df['cell_count'].unique())
            cell_count = df['cell_count'].unique()[0]
            num_cells = num_cells + cell_count
            num_dots = num_dots + df[df['dot_area']>dot_area_thres]['dot_area'].count()
            
        summary = summary.append([{'number of dots': num_dots,
                                   'number of cells': num_cells,
                            'dot_area_thres': dot_area_thres,
                                'dot_color' : dot_col,
                                'field' :  field,
                                'treatment' :  condition,
                             },])

summary = summary.reset_index()
#         print('Treatment :', condition, '\n',
#               'dot color :', dot_col, '\n'
#               'number of cells :', num_cells, '\n', 
#               'number of dots :', num_dots, '\n',
#              'average dots per cells :', num_dots/num_cells, '\n')

In [19]:
summary.to_excel(os.path.join(os.path.dirname(seg_dir),'Segmentation_'+ str(erosion_radius)
                       + '_min_area_' + str(min_area_to_keep_cell) +'_area_thresh_' + str(area_thresh)+'.xlsx'))

In [20]:
summary_cell = pd.DataFrame()
for condition in conditions:
    cell_num = 0
    
    for field in summary_df['field'].unique():
        df = summary_df[(summary_df['field'] == field) & 
                            (summary_df['treatment'] == condition)]
        
        cell_count = df['cell_count'].unique()[0]
        
        for count, cell in enumerate(df['cell_id'].unique()):
            cell_num = cell_num + 1
            
            n_green = len(df[(df['dot_color'] == 'green') & (df['dot_area']>0) & (df['cell_id'] == cell)])
            n_red = len(df[(df['dot_color'] == 'red') & (df['dot_area']>4) & (df['cell_id'] == cell)])
            n_yellow = len(df[(df['dot_color'] == 'yellow') & (df['dot_area']>4) & (df['cell_id'] == cell)])
            
            summary_cell = summary_cell.append([{'num_green_dots': n_green,
                                                 'num_red_dots': n_red,
                                                 'num_yellow_dots': n_yellow,
                                                      'cell_num': cell_num,
                                                 'treatment' :  condition,
                                                         },])

            
        while count < cell_count:
            count = count + 1
            n_green,n_red, n_yellow = 0, 0, 0

            
            


summary_cell = summary_cell.reset_index()


In [21]:
summary_cell.to_excel(os.path.join(os.path.dirname(seg_dir),'Summary_'+ str(erosion_radius)
                       + '_min_area_' + str(min_area_to_keep_cell) +'_area_thresh_' + str(area_thresh)+'.xlsx'))

In [22]:
list(summary_cell)

['index',
 'cell_num',
 'num_green_dots',
 'num_red_dots',
 'num_yellow_dots',
 'treatment']

In [26]:
import plotly_express as px
from plotly.offline import plot 

In [27]:
fig = px.histogram(summary_cell, x="num_yellow_dots", color='treatment')
plot(fig)

'temp-plot.html'

# Scratch Space